In [1]:
import mxnet as mx 
import numpy as np 
from Symbol import symbol

In [2]:
import mxnet as mx

In [3]:
import numpy as np 
a = np.ones(shape=(3,2))
b = np.ones(shape=(1,2))*2
print a*b
print a
print np.mean(a*b)

[[ 2.  2.]
 [ 2.  2.]
 [ 2.  2.]]
[[ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]]
2.0


In [4]:
a[:, np.newaxis].shape

(3, 1, 2)

In [15]:
model_path="/home/ec2-user/yolo/drive_full_detect" 
epoch=0

In [16]:
sym, args, aux = mx.model.load_checkpoint(model_path, epoch)
# extract last bn layer
sym = sym.get_internals()['bn1_output']

In [17]:
anchors = np.asarray([[ 449.61853027,  360.13400269],
       [ 234.92941284,  241.6000061 ],
       [ 120.71929932,  115.58772278],
       [ 302.77478027,  403.26126099],
       [ 395.41271973,  236.67462158]], dtype=np.float32)
num_boxes = len(anchors)

In [18]:
sym = mx.sym.Activation(data=sym, act_type="relu", name="relu_final")
sym = mx.sym.Convolution(data=sym, kernel=(3, 3),
                         num_filter=num_boxes*5, pad=(1, 1),
                         stride=(1, 1), no_bias=True,
                         )
sym = mx.sym.transpose(sym, (0,2,3,1))
# get softsign
sym = sym / (1 + mx.sym.abs(sym))

In [19]:
def YoloV2loss(label, pred, anchors):
    pred_splits = mx.sym.split(splits, num_outputs=num_boxes*5, axis=2) 
    cp1, xp1, yp1, wp1, hp1, \
    cp2, xp2, yp2, wp2, hp2, \
    cp3, xp3, yp3, wp3, hp3, \
    cp4, xp4, yp4, wp4, hp4, \
    cp5, xp5, yp5, wp5, hp5= mx.sym.split(pred, num_outputs=25, axis=2)    
    cl, xl, yl, wl, hl = mx.sym.split(label, num_outputs=5, axis=2)
    
    
    if_wsmall = (cl - anchors[0][0])>0
    if_hsmall = (cl - anchors[0][1]>0)
    intersect = (wl*(if_wsmall)+anchors[0][0]*(1-if_wsmall)) * (hl*(if_hsmall)+anchors[0][1]*(1-if_hsmall))
    IOU = intersect / (wl*hl+anchors[0][0]*anchors[0][1] - intersect)

In [20]:
t1 = mx.nd.array(ctx=mx.cpu(), source_array=np.asarray([1,20,3,4,45]))
t2 = mx.nd.array(ctx=mx.cpu(), source_array=np.asarray([1,2,13,4,5]))

In [21]:
t1[t1>t2]+t2[t1<t2]

ValueError: NDArray does not support slicing with key 
[ 0.  1.  0.  0.  1.]
<NDArray 5 @cpu(0)> of type <class 'mxnet.ndarray.NDArray'>.